In [1]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from sklearn.model_selection import train_test_split
import torch
import pandas as pd

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
checkpoint = "../../data/classification/models/detector-base.pt"
data = torch.load(checkpoint, map_location=device)

In [4]:
model_name = "roberta-base"
model = RobertaForSequenceClassification.from_pretrained(model_name)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [5]:
model.load_state_dict(data['model_state_dict'],strict=False)
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [6]:
device="cuda"
query = """I liked nothing about this dress. The only reason I gave it 4 stars is because I ordered the size 6.5 and it fit perfectly. The fabric is a nice thin material and the color is vibrant. It's a little thin on me but it's worth it. The material is soft and comfortable. I love the color and the fit. I'm 5'4" and it fits just right. I'm a size 6 and it's a bit long on me so I ordered a 6 in order to fit the smaller size. I'm 5'3" and it fits perfect and I got it in a medium. It's a nice dress and I'm very pleased with it. I like that it's a bit small for me and I can wear it with a top or skirt. I also like that the fabric is made of a good material. I wear a size 5 and it fits well. I'm 5'3" and it fits just right. I'm 5'6" and it fits just right. I'm 5'4" and it fits just right. I wear a size 10.5 and it's a little tight on me. I have a small waist and it's not too tight on me. It's a nice little dress. I will buy more colors.Very nice dress.  It is well made and it is very comfortable.  I received a free sample and will see if I can wear it again.  I would definitely buy another one.  I am very happy with this dress.I love these boots!  I have been wearing them for almost 8 months now and I love them so much that I purchased a second pair.  They are very comfortable and look great. """
tokens = tokenizer.encode(query)
all_tokens = len(tokens)
tokens = tokens[:tokenizer.model_max_length - 2]
used_tokens = len(tokens)
tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}\nAll Tokens: {all_tokens}\nUsed Tokens: {used_tokens}")

Real Probability: 0.0002431118191452697
Fake Probability: 0.9997568726539612
All Tokens: 351
Used Tokens: 351


In [7]:
device="cuda"
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
tokens = tokenizer.encode(query)
all_tokens = len(tokens)
tokens = tokens[:tokenizer.model_max_length - 2]
used_tokens = len(tokens)
tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
mask = torch.ones_like(tokens)

with torch.no_grad():
    logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
    probs = logits.softmax(dim=-1)

fake, real = probs.detach().cpu().flatten().numpy().tolist()

print(f"Real Probability: {real}\nFake Probability: {fake}\nAll Tokens: {all_tokens}\nUsed Tokens: {used_tokens}")

Real Probability: 0.9997749924659729
Fake Probability: 0.00022505233937408775
All Tokens: 408
Used Tokens: 408


In [8]:
encoded_label_dict = {"CG" : 0, "OR" : 1}
def encode_label(x):
    return encoded_label_dict.get(x,-1)

In [9]:
df = pd.read_csv("../../data/classification/data/data.csv")

In [10]:
df["target"] = df["label"].apply(lambda x: encode_label(x))

In [11]:
train_df, valid_df = train_test_split(df, test_size=0.2, shuffle=True, stratify=None, random_state=2021)

In [12]:
train_df.shape, valid_df.shape

((32345, 5), (8087, 5))

In [13]:
train_df.label.value_counts()

CG    16206
OR    16139
Name: label, dtype: int64

In [14]:
train_df.category.value_counts()

Kindle_Store_5                  3812
Books_5                         3492
Pet_Supplies_5                  3410
Home_and_Kitchen_5              3264
Electronics_5                   3216
Sports_and_Outdoors_5           3114
Clothing_Shoes_and_Jewelry_5    3075
Tools_and_Home_Improvement_5    3061
Toys_and_Games_5                3040
Movies_and_TV_5                 2861
Name: category, dtype: int64

In [15]:
valid_df.label.value_counts()

OR    4077
CG    4010
Name: label, dtype: int64

In [16]:
valid_df.category.value_counts()

Kindle_Store_5                  918
Books_5                         878
Pet_Supplies_5                  844
Sports_and_Outdoors_5           832
Tools_and_Home_Improvement_5    797
Home_and_Kitchen_5              792
Clothing_Shoes_and_Jewelry_5    773
Electronics_5                   772
Toys_and_Games_5                754
Movies_and_TV_5                 727
Name: category, dtype: int64

In [17]:
def predict(query, model, tokenizer, device="cuda"):
    tokens = tokenizer.encode(query)
    all_tokens = len(tokens)
    tokens = tokens[:tokenizer.model_max_length - 2]
    used_tokens = len(tokens)
    tokens = torch.tensor([tokenizer.bos_token_id] + tokens + [tokenizer.eos_token_id]).unsqueeze(0)
    mask = torch.ones_like(tokens)

    with torch.no_grad():
        logits = model(tokens.to(device), attention_mask=mask.to(device))[0]
        probs = logits.softmax(dim=-1)

    fake, real = probs.detach().cpu().flatten().numpy().tolist()
    return real

In [18]:
query = """I work in the wedding industry and have to work long days, on my feet, outside in the heat, and have to look professional. I've spent a ridiculous amount of money on high end dress shoes like Merrels and just have not been able to find a pair that are comfortable to wear all day. Both for my feet and my back. Enter the Sanuk yoga sling!!! These shoes are amazingly comfortable. Though, I will admit it took a few wears to get used to the feel of the yoga matte bottom. At first, it felt a little "sticky" to me, and the fabric part that goes through the toe area was a little thick and took some getting used to. I wore them for a few days before taking them out on a job and I can't get over how comfortable they are. Ii have been wearing these shoes now for 3 months, every work day and I am THRILLED. No more back pain, no more sore feet. I also wear these sometimes during my off time,mans every time I wear them, I get compliments on how cute and comfortable they look. The great thing about these shoes is the yoga matte bottom. It helps your feet grip to the shoe a bit, so your foot can just walk normally, without having to grip the shoe. You may not realize it, but with a lot of Sandals, your foot is having to work to keep the shoe on, changing the way you walk and stand and ultimately causing foot and back pain. Not with these! Also, the soft linen sits comfortably on your skin and breathes nicely in the heat. The only downside is the funky tan lines, which is why I am sure to alternate shoes on my days off, especially if I plan to be outside for most of the day. If it were not for that, I think these might be the only shoes I'd wear all summer. If you are looking for a reasonable priced, comfortable shoe that you can wear and walk in all day."""
predict(query,model,tokenizer)

0.9997749924659729

In [28]:
preds, preds_probas = [],[]
for i, row in valid_df.iterrows():
    query = row["text_"]
    pred = predict(query,model,tokenizer)
    preds_probas.append(pred)
    if pred >= 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [20]:
from sklearn.metrics import confusion_matrix
y_true = valid_df.target.values
y_pred = preds
confusion_matrix(y_true,y_pred)

array([[3700,  310],
       [1143, 2934]])

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
acc = accuracy_score(y_true,y_pred)
precision = precision_score(y_true,y_pred)
recall = recall_score(y_true,y_pred)

In [22]:
print(f"Accuracy: {acc*100}; Precision:{precision*100}; Recall:{recall*100}")

Accuracy: 82.03289229627798; Precision:90.44389642416769; Recall:71.96467991169978


#### Writing predictions to disc

In [29]:
preds_df_rows = []
for i, row in valid_df.reset_index().iterrows():
    query = row["text_"]
    pred_prob = preds_probas[i]
    pred_label = preds[i]
    preds_df_rows.append([pred_prob,pred_label])
preds_df = pd.DataFrame(preds_df_rows, columns=["GPT2_Detector_Model_Probability","GPT2_Detector_Model_Prediction"])

In [30]:
preds_df.to_csv("../../data/classification/data/gpt2_detector_predictions.csv", index=None)